# Trabajo final - Organizar los horarios de partidos de La Liga
## Daniel Velásquez

### Repositorio: https://github.com/dfelipe/MIA_AO_Trabajo.final

Descripción del problema: Desde la La Liga de fútbol profesional se pretende organizar los horarios de los partidos de liga de cada jornada. Se conocen algunos datos que nos deben llevar a diseñar un algoritmo que realice la asignación de los partidos a los horarios de forma que maximice la audiencia.

#### (*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?
Considerando que en cada uno de los 10 horarios se pueden presentar hasta 8 coincidencias (9 partidos en simultaneo), la combinación sería el resultado de los 90 espacios disponibles (10 horarios x 9 partidos por horario), para lo cual se deben realizar permutaciones sin repeticiones (cada partido es único): 

La fórmula para la variación con repeticiones es V(n, r) = n^r (9 partidos simultaneos con 10 partidos definidos) y considerando que existen 10 posibles horarios:

In [504]:
print("Posibilidades sin considerar restricciones: ",(9**10)*10)

Posibilidades sin considerar restricciones:  34867844010


#### ¿Cuántas posibilidades hay teniendo en cuenta todas las restricciones?

Las restricciones definidas en el problema indican que el viernes a las 20:00Hrs y lunes a las 20:00Hrs deben tener al menos un partido cada horario, por lo cual se debe separar el problema en dos, la primera es encontrar la combinación de los 10 partidos en los 2 horarios fijos:

C(10, 2) = 10! / (2! * (10 - 2)!)= 45

Y la segunda parte es la variación de los 8 partidos restantes.

El resultado total es la multiplicación de las dos partes.

In [505]:
print("Posibilidades considerando restricciones: ",(9**8)*10*45)

Posibilidades considerando restricciones:  19371024450


#### (*) ¿Cuál es la estructura de datos que mejor se adapta al problema? Argumenta la respuesta
(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, argumenta)

Generé múltiples matrices para poder almacenar: (1) los equipos, (2) partidos con su audiencia, (3) Audiencia según coincidencia, (4) combinación de partidos por horario.

La estructura (4) es la que utilicé para representar las diferentes combinaciones que se pueden dar de los partidos en los múltiples horarios con o sin coincidencia. La esttrucutra es una matriz de 10x9 (10 horarios por 9 potenciales partidos al mismo tiempo) donde en cada ubicación se puede tener o no el identificador de un partido (ej: 3 - Athletic vs. Mallorca). 

Ej:

[ 9.,0., 0., 0.,10., 0., 0., 0., 1., 2.]  
[0., 0., 5., 0., 0., 0., 0., 0., 0., 0.]  
[0., 7., 0., 0., 0., 0., 0., 0., 0., 0.]  
[0., 0., 3., 0., 0., 0., 0., 0., 0., 0.]  
[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]  
[0., 0., 0., 0., 0., 0., 6., 0., 0., 0.]  
[0., 0., 0., 0., 0., 0., 4., 0., 0., 0.]  
[0., 0., 0., 0., 0., 8., 0., 0., 0., 0.]  
[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]  
[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]  

Esta estructura conecta con las otras matrices para posteriormente poder realizar el cálculo según coincidencias por horario (columna) y tipo de partido (id de cada celda).

Realizar las posibles combinaciones es factible con esta estructura porque se puede desplegar los 10 partidos definidos en toda la matriz y además permite tener más de 10 partidos, con un máximo de 100 partidos incluso.

Inicialmente la estructura utilizada al principio solo tenía un vector para los partidos por día, pero se debía considerar múltiples vectores por día, lo que compliejizaba el manejo y algoritmo para el cálculo de audiencia.

#### (*)¿Cuál es la función objetivo?

La función objetivo es encontrar la audiencia de mayor tamaño considerando un grupo de 10 partidos ya definidos con su respectiva audiencia, en 10 horarios diferentes con la posibilidad de tener hasta 8 partidos concurrentes.

#### (*)¿Es un problema de maximización o minimización?

Es un problema de maximización, donde se debe buscar la forma de encontrar la combinación que represente la mayor cantidad de audiencia posible considerando las restricciones de tener un partido el viernes a las 20:00Hrs y otro el lunes a las 20:00Hrs.

#### (*)Diseña un algoritmo que mejore la complejidad del algoritmo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

El algoritmo que se presenta a continuación se desarrolló con dos lineamientos diferentes, el primero buscando la distribución de partidos considerando un partido por horario y que cumple con la restricción de un partido el viernes y lunes a las 20:00Hrs. El resultado es una distribución con una audiencia de **6.856.000** de personas en total.

El segundo buscó una distribución considerando la posibilidad de tener conicidencias de hasta 2 partidos por horario y con la restricción de tener un partido el viernes y el lunes a las 20:00Hrs. En este caso el mejor resultado encontrado es una distribución con audiencia en total de **6.330.000** de personas.

El análisis de los resultados y de los datos entregados con respecto a la penalización de audiencia cuando hay coincidencias en un horario, indica que no hay incentivos para realizar más de un partido en un mismo horario si el número de partidos coincide con el número de horarios. Este ejercicio plantea ese escenario.

Por otra parte, si el número de partidos a disputar supera la cantidad de horarios disponible, este algoritmo desarrollado permite encontrar una distribución con el máximo de audiencia disponible.

#### (*)Calcula la complejidad del algoritmo

Para la búsqueda de combinaciones de un partido por horario se utilizó la librería "itertools.permutations", la cual entrega para el algortimos desarrollado, todas las posibles distribuciones que hay con un partido por horario (total **3628800**).

Este listado se utiliza posteriormente para buscar distribuciones alternativas con partidos concurrentes.

La complejidad del algoritmo se debe separar en varias partes: 

(1) Las permutaciones tienen una complejidad de **n!**
(2) Nuevas distribuciones con coincidencias realiza una iteración por el total encontrado previamente **n!**
(3) La función objetivo realiza un recorrido calculando la suma de audiencia **n!**

En total se debería realizar una suma (1), (2) y (3), con lo cual en terminos generales, la complejidad del algortimo es de **n!**.

### Desarrollo del algoritmo:

In [1]:
import numpy as np
import random 
import copy
import itertools

In [507]:
#Estructuras de datos para los equipos, partidos y horarios
teams=np.array([[1,"Real Madrid",1],[2,"Real Sociedad",1],[3,"Barcelona",1],[4,"Celta",2],[5,"Valencia",2],[6,"Athletic",2],[7,"Villareal",2],[8,"Alavés",2],[9,"Levante",2],[10,"Espannyol",2],[11,"Sevilla",2],[12,"Betis",2],[13,"Atlético",2],[14,"Getafe",2],[15,"Mallorca",3],[16,"Eibar",3],[17,"Leganés",3],[18,"Osasuna",3],[19,"Granada",3],[20,"Valladolid",3]])
days=np.array([0.4,0.55,0.7,0.8,1,0.45,0.75,0.85,1,0.4])
days_name=np.array(["V20","S12","S16","S18","S20","D12","D16","D18","D20","L20"])

print(teams)

#Se generan los partidos establecidos en el ejercicio con la audiencia de cada uno
matchs=[]
matchs.extend([[teams[0],teams[3],1.3]])
matchs.extend([[teams[4],teams[1],1.3]])
matchs.extend([[teams[14],teams[15],0.47]])
matchs.extend([[teams[5],teams[2],1.3]])
matchs.extend([[teams[16],teams[17],0.47]])
matchs.extend([[teams[6],teams[18],0.75]])
matchs.extend([[teams[7],teams[8],0.9]])
matchs.extend([[teams[9],teams[10],0.9]])
matchs.extend([[teams[11],teams[19],0.75]])
matchs.extend([[teams[12],teams[13],0.9]])

print("Partidos: ")
for x in range(10):
        print(matchs[x][0][1], " vs. ", matchs[x][1][1], " aud: ",matchs[x][2] )

[['1' 'Real Madrid' '1']
 ['2' 'Real Sociedad' '1']
 ['3' 'Barcelona' '1']
 ['4' 'Celta' '2']
 ['5' 'Valencia' '2']
 ['6' 'Athletic' '2']
 ['7' 'Villareal' '2']
 ['8' 'Alavés' '2']
 ['9' 'Levante' '2']
 ['10' 'Espannyol' '2']
 ['11' 'Sevilla' '2']
 ['12' 'Betis' '2']
 ['13' 'Atlético' '2']
 ['14' 'Getafe' '2']
 ['15' 'Mallorca' '3']
 ['16' 'Eibar' '3']
 ['17' 'Leganés' '3']
 ['18' 'Osasuna' '3']
 ['19' 'Granada' '3']
 ['20' 'Valladolid' '3']]
Partidos: 
Real Madrid  vs.  Celta  aud:  1.3
Valencia  vs.  Real Sociedad  aud:  1.3
Mallorca  vs.  Eibar  aud:  0.47
Athletic  vs.  Barcelona  aud:  1.3
Leganés  vs.  Osasuna  aud:  0.47
Villareal  vs.  Granada  aud:  0.75
Alavés  vs.  Levante  aud:  0.9
Espannyol  vs.  Sevilla  aud:  0.9
Betis  vs.  Valladolid  aud:  0.75
Atlético  vs.  Getafe  aud:  0.9


In [364]:
#Función para generar permutaciones con un partido por horario al tiempo
def fb_com(match):
    comb=list(itertools.permutations(match, len(match)))
    return comb

fb_teams=[0,1,2,3,4,5,6,7,8,9]

teams_t=fb_com(fb_teams)

print("Permutaciones: ",len(teams_t))

Permutaciones:  3628800


In [506]:
#Función objetivo que permite identificar la audiencia y retorna la de mayor volumen conun partido por día
def fitness(matchs,fb_total, days):
    best_audi=0
    best_pos=0
    for x in range(len(fb_total)):
        summ=0
        for y in range(len(fb_total[x])):
            summ=summ+matchs[fb_total[x][y]][2]*days[y]
        if summ>best_audi:
            best_audi=summ
            best_pos=x
    return best_pos, best_audi

In [509]:
#Cálculo de audiencia considerando las 3.6 millones de combinaciones sin coincidencia de partidos
best_sol_1m, best_audi_1m= fitness(matchs,teams_t,days)

print(teams_t[best_sol_1m])
print("Mejor audiencia: ", round(best_audi_1m,3))
print("Partidos: ")
for x in range(10):
        print(matchs[teams_t[best_sol_1m][x]][0][1], " vs. ", matchs[teams_t[best_sol_1m][x]][1][1], " H: ", days_name[x])

(2, 5, 6, 7, 0, 8, 9, 1, 3, 4)
Mejor audiencia:  6.856
Partidos: 
Mallorca  vs.  Eibar  H:  V20
Villareal  vs.  Granada  H:  S12
Alavés  vs.  Levante  H:  S16
Espannyol  vs.  Sevilla  H:  S18
Real Madrid  vs.  Celta  H:  S20
Betis  vs.  Valladolid  H:  D12
Atlético  vs.  Getafe  H:  D16
Valencia  vs.  Real Sociedad  H:  D18
Athletic  vs.  Barcelona  H:  D20
Leganés  vs.  Osasuna  H:  L20


**Una distribución con el máximo de audiencia posible considerando un partido por horario:**  

Mejor audiencia:  6.856
Partidos: 
Mallorca  vs.  Eibar  H:  V20
Villareal  vs.  Granada  H:  S12
Alavés  vs.  Levante  H:  S16
Espannyol  vs.  Sevilla  H:  S18
Real Madrid  vs.  Celta  H:  S20
Betis  vs.  Valladolid  H:  D12
Atlético  vs.  Getafe  H:  D16
Valencia  vs.  Real Sociedad  H:  D18
Athletic  vs.  Barcelona  H:  D20
Leganés  vs.  Osasuna  H:  L20

In [372]:
#Genera una matriz con el porcentaje de audiencia según coincidencias en cada horario
def mat_days(days):
    shape = (8, 10)
    m_days= np.zeros(shape)
    
    for x in range(10):
        m_days[0][x]=days[x]
    
    for y in range(8):
        for z in range(10):
            if y==1:
                m_days[y][z]=m_days[0][z]*0.75
            if y==2:
                m_days[y][z]=m_days[0][z]*0.65
            if y==3:
                m_days[y][z]=m_days[0][z]*0.4
            if y==4:
                m_days[y][z]=m_days[0][z]*0.3
            if y==5:
                m_days[y][z]=m_days[0][z]*0.25
            if y==6:
                m_days[y][z]=m_days[0][z]*0.22
            if y==7:
                m_days[y][z]=m_days[0][z]*0.2
            if y==8:
                m_days[y][z]=m_days[0][z]*0.2
            
    
    print(m_days)
    return m_days

m_days=mat_days(days)

[[0.4    0.55   0.7    0.8    1.     0.45   0.75   0.85   1.     0.4   ]
 [0.3    0.4125 0.525  0.6    0.75   0.3375 0.5625 0.6375 0.75   0.3   ]
 [0.26   0.3575 0.455  0.52   0.65   0.2925 0.4875 0.5525 0.65   0.26  ]
 [0.16   0.22   0.28   0.32   0.4    0.18   0.3    0.34   0.4    0.16  ]
 [0.12   0.165  0.21   0.24   0.3    0.135  0.225  0.255  0.3    0.12  ]
 [0.1    0.1375 0.175  0.2    0.25   0.1125 0.1875 0.2125 0.25   0.1   ]
 [0.088  0.121  0.154  0.176  0.22   0.099  0.165  0.187  0.22   0.088 ]
 [0.08   0.11   0.14   0.16   0.2    0.09   0.15   0.17   0.2    0.08  ]]


In [373]:
#Genera una solución (matriz) con coincidencias en días a partir de una solución sin coincidencias (1D)
def mat_sol(teams):
    shape = (10, 10)
    m_sol= np.zeros(shape)
    tt=copy.deepcopy(teams)
    if tt[0]==0:
        m_sol[0][0]=10
    else:
        m_sol[0][0]=tt[0]
    if tt[9]==0:
        m_sol[0][9]=10
    else:
        m_sol[0][9]=tt[9]
    for x in range(1,9):
        if tt[x]==0:
            m_sol[x][np.random.randint(1,8)]=10
        else:
            m_sol[x][np.random.randint(1,8)]=tt[x]
    return m_sol

In [497]:
#Función objetivo que procesa matrices que permite identificar la audiencia y retorna la de mayor volumen
def mat_fitness(matchs,m_sol, m_days):
    best_audi=0
    copy_m_sol=copy.deepcopy(m_sol)
    summ=0
    hh=np.count_nonzero(copy_m_sol != 0, axis=0)
    for x in range(len(copy_m_sol)):       
        summ=0
        for y in range(len(copy_m_sol)):
            if copy_m_sol[x][y]==10:
                summ=summ+matchs[0][2]*m_days[int(hh[x])-1][y]
    #            print("a:",matchs[0][2], " b:", m_days[int(hh[x])-1][y])
            elif m_sol[x][y]!=0:
                summ=summ+matchs[int(copy_m_sol[x][y])][2]*m_days[int(hh[x])-1][y]
    #            print("a:",matchs[int(copy_m_sol[x][y])][0], " t:" ,matchs[int(copy_m_sol[x][y])][2], " b:", m_days[int(hh[x])-1][y])
        if summ>best_audi:
            best_audi=summ
    return best_audi

In [378]:
#Obtiene la audiencia desde el listado de 3.6MM sin aplicar criterios de selección
def search_comb(matchs,teams_t, m_days):
    best_solution=[]
    best_audi=0
    for ff in range(3527799, 3627799):
        m_sol=mat_sol(teams_t[ff])
        tem_audi= mat_fitness(matchs,m_sol,m_days)
        if tem_audi>best_audi:
            best_solution=copy.deepcopy(m_sol)
            best_audi=tem_audi
    return best_solution, best_audi

In [511]:
#Obtiene la audiencia desde el listado de 3.6MM aplicando criterios de selección
def search_gen(matchs,teams_t, m_days):
    best_solution=[]
    best_audi=0
    for ff in range(1, len(teams_t)):
        m_sol=mat_sol_gen(teams_t[ff])
        tem_audi= mat_fitness(matchs,m_sol,m_days)
        if tem_audi>best_audi:
            best_solution=copy.deepcopy(m_sol)
            best_audi=tem_audi
    return best_solution, best_audi

In [380]:
#Genera una solución (matriz) con coincidencias en días de mejor audiencia a partir de una solución sin coincidencias (1D)
def mat_sol_gen(teams):
    shape = (10, 10)
    m_sol= np.zeros(shape)
    tt=copy.deepcopy(teams)
    if tt[0]==0:
        m_sol[0][0]=10
    else:
        m_sol[0][0]=tt[0]
    if tt[4]==0:
        m_sol[0][4]=10
    else:
        m_sol[0][4]=tt[4]
    if tt[8]==0:
        m_sol[0][8]=10
    else:
        m_sol[0][8]=tt[8]
    if tt[9]==0:
        m_sol[0][9]=10
    else:
        m_sol[0][9]=tt[9]
    for x in range(1,4):
        if tt[x]==0:
            m_sol[x][np.random.randint(1,3)]=10
        else:
            m_sol[x][np.random.randint(1,3)]=tt[x]
    for x in range(5,8):
        if tt[x]==0:
            m_sol[x][np.random.randint(5,7)]=10
        else:
            m_sol[x][np.random.randint(5,7)]=tt[x]
    return m_sol

In [448]:
#Busca las mejores soluciones considerando combinaciones en el día y fijando 4 horarios(V,S,D,L)
def search_gen_1(matchs,teams_t, m_days):
    best_solution= []
    best_audi=[]
    max_audi=5
    for ff in range(1, 3627799):
        m_sol=mat_sol_gen(teams_t[ff])
        tem_audi= mat_fitness(matchs,m_sol,m_days)
        if tem_audi>max_audi:
            best_solution.append(list(m_sol))
            best_audi.append(tem_audi)
            max_audi=tem_audi
    return best_solution, best_audi

In [449]:
#Muestra el resultado de la mejor distribución con la audiencia total
best_solution, best_audi=search_gen_1(matchs,teams_t, m_days)

print("Total de combinaciones mayores a 5MM: ", len(best_solution))

for z in range(10):
    for x in range(10):
        for y in range(10):
            if best_solution[len(best_solution)-z-1][x][y]==10:
                print(matchs[0][0][1], " vs. ", matchs[0][1][1], " H: ", days_name[y])
            elif best_solution[len(best_solution)-z-1][x][y]!=0:
                print(matchs[int(best_solution[len(best_solution)-z-1][x][y])][0][1], " vs. ", matchs[int(best_solution[len(best_solution)-z-1][x][y])][1][1], " H: ", days_name[y])
    print("Audiencia: ", round(best_audi[len(best_solution)-z-1],2))

Total de combinaciones mayores a 5MM:  21
Betis  vs.  Valladolid  H:  V20
Valencia  vs.  Real Sociedad  H:  S20
Real Madrid  vs.  Celta  H:  D20
Villareal  vs.  Granada  H:  L20
Alavés  vs.  Levante  H:  S16
Espannyol  vs.  Sevilla  H:  S16
Leganés  vs.  Osasuna  H:  S12
Athletic  vs.  Barcelona  H:  D16
Atlético  vs.  Getafe  H:  D16
Mallorca  vs.  Eibar  H:  D12
Audiencia:  5.88
Villareal  vs.  Granada  H:  V20
Athletic  vs.  Barcelona  H:  S20
Real Madrid  vs.  Celta  H:  D20
Espannyol  vs.  Sevilla  H:  L20
Alavés  vs.  Levante  H:  S16
Betis  vs.  Valladolid  H:  S16
Mallorca  vs.  Eibar  H:  S12
Valencia  vs.  Real Sociedad  H:  D16
Atlético  vs.  Getafe  H:  D16
Leganés  vs.  Osasuna  H:  D12
Audiencia:  5.86
Villareal  vs.  Granada  H:  V20
Real Madrid  vs.  Celta  H:  S20
Valencia  vs.  Real Sociedad  H:  D20
Alavés  vs.  Levante  H:  L20
Athletic  vs.  Barcelona  H:  S16
Betis  vs.  Valladolid  H:  S16
Mallorca  vs.  Eibar  H:  S12
Espannyol  vs.  Sevilla  H:  D16
Atlético  v

In [468]:
#Genera una solución (matriz) con coincidencias en días a partir de una solución sin coincidencias (1D)
def mat_sol_prob(teams):
    shape = (10, 10)
    m_sol= np.zeros(shape)
    tt=copy.deepcopy(teams)
    if tt[0]==0:
        m_sol[0][0]=10
    else:
        m_sol[0][0]=tt[0]
    if tt[1]==0:
        m_sol[0][2]=10
    else:
        m_sol[0][2]=tt[1]
    if tt[2]==0:
        m_sol[0][3]=10
    else:
        m_sol[0][3]=tt[2]
    if tt[3]==0:
        m_sol[0][4]=10
    else:
        m_sol[0][4]=tt[3]        
    if tt[4]==0:
        m_sol[1][4]=10
    else:
        m_sol[1][4]=tt[4]
    if tt[5]==0:
        m_sol[0][6]=10
    else:
        m_sol[0][6]=tt[5]
    if tt[6]==0:
        m_sol[0][7]=10
    else:
        m_sol[0][7]=tt[6]
    if tt[7]==0:
        m_sol[0][8]=10
    else:
        m_sol[0][8]=tt[7]
    if tt[8]==0:
        m_sol[1][8]=10
    else:
        m_sol[1][8]=tt[8]
    if tt[9]==0:
        m_sol[0][9]=10
    else:
        m_sol[0][9]=tt[9]
 #   print(m_sol)
    return m_sol

In [498]:
#Busca las mejores soluciones considerando combinaciones en el día ajustando en varios horarios
def search_ajust(matchs,teams_t, m_days):
    best_solution= []
    best_audi=[]
    max_audi=5
    for ff in range(1, 3627799):
        m_sol=mat_sol_prob(teams_t[ff])
        tem_audi= mat_fitness(matchs,m_sol,m_days)
        if tem_audi>max_audi:
            best_solution.append(list(m_sol))
            best_audi.append(tem_audi)
            max_audi=tem_audi
    return best_solution, best_audi

In [499]:
#Muestra el resultado de la mejor distribución con la audiencia total
best_solution, best_audi=search_ajust(matchs,teams_t, m_days)


In [510]:
print("Total de combinaciones mayores a 5MM: ", len(best_solution))

for z in range(len(best_solution)):
    for x in range(10):
        for y in range(10):
            if best_solution[len(best_solution)-z-1][x][y]==10:
                print(matchs[0][0][1], " vs. ", matchs[0][1][1], " H: ", days_name[y])
            elif best_solution[len(best_solution)-z-1][x][y]!=0:
                print(matchs[int(best_solution[len(best_solution)-z-1][x][y])][0][1], " vs. ", matchs[int(best_solution[len(best_solution)-z-1][x][y])][1][1], " H: ", days_name[y])
    print("Audiencia: ", round(best_audi[len(best_solution)-z-1],4))

Total de combinaciones mayores a 5MM:  19
Villareal  vs.  Granada  H:  V20
Alavés  vs.  Levante  H:  S16
Espannyol  vs.  Sevilla  H:  S18
Real Madrid  vs.  Celta  H:  S20
Atlético  vs.  Getafe  H:  D16
Valencia  vs.  Real Sociedad  H:  D18
Athletic  vs.  Barcelona  H:  D20
Betis  vs.  Valladolid  H:  L20
Mallorca  vs.  Eibar  H:  S20
Leganés  vs.  Osasuna  H:  D20
Audiencia:  6.33
Villareal  vs.  Granada  H:  V20
Alavés  vs.  Levante  H:  S16
Real Madrid  vs.  Celta  H:  S18
Valencia  vs.  Real Sociedad  H:  S20
Espannyol  vs.  Sevilla  H:  D16
Atlético  vs.  Getafe  H:  D18
Athletic  vs.  Barcelona  H:  D20
Betis  vs.  Valladolid  H:  L20
Mallorca  vs.  Eibar  H:  S20
Leganés  vs.  Osasuna  H:  D20
Audiencia:  6.31
Villareal  vs.  Granada  H:  V20
Real Madrid  vs.  Celta  H:  S16
Alavés  vs.  Levante  H:  S18
Valencia  vs.  Real Sociedad  H:  S20
Espannyol  vs.  Sevilla  H:  D16
Atlético  vs.  Getafe  H:  D18
Athletic  vs.  Barcelona  H:  D20
Betis  vs.  Valladolid  H:  L20
Mallorca  

In [512]:
#Mezcla las mejores combinaciones encontradas
def mut_sol(best_solution):
    bb=copy.deepcopy(best_solution)
    m_audi=[]
    m_sol=[]
    for x in range(len(bb)):
        bb_m=np.array(bb[x])
       # print(bb_m)
        p=random.randint(1,4)
        t=random.randint(5,8)
        te=copy.deepcopy(bb_m)
       # print(te)
        bb_m[:,p]=bb_m[:,t]
       # print(bb_m)
        bb_m[:,t]=te[:,p]
       # print(bb_m)
        m_sol.append(bb_m)
      #  print(m_sol)
        m_audi.append(mat_fitness(matchs,bb_m,m_days))
      #  print(m_audi)
    return m_sol, m_audi

m_solutions, m_audi=mut_sol(best_solution)

for z in range(len(m_solutions)):
    for x in range(10):
        for y in range(10):
            if m_solutions[len(m_solutions)-z-1][x][y]==10:
                print(matchs[0][0][1], " vs. ", matchs[0][1][1], " H: ", days_name[y])
            elif m_solutions[len(m_solutions)-z-1][x][y]!=0:
                print(matchs[int(m_solutions[len(m_solutions)-z-1][x][y])][0][1], " vs. ", matchs[int(m_solutions[len(m_solutions)-z-1][x][y])][1][1], " H: ", days_name[y])
    print("Audiencia: ", round(m_audi[len(m_audi)-z-1],2))



Villareal  vs.  Granada  H:  V20
Valencia  vs.  Real Sociedad  H:  S16
Espannyol  vs.  Sevilla  H:  S18
Real Madrid  vs.  Celta  H:  S20
Atlético  vs.  Getafe  H:  D16
Alavés  vs.  Levante  H:  D18
Athletic  vs.  Barcelona  H:  D20
Betis  vs.  Valladolid  H:  L20
Mallorca  vs.  Eibar  H:  S20
Leganés  vs.  Osasuna  H:  D20
Audiencia:  6.27
Villareal  vs.  Granada  H:  V20
Alavés  vs.  Levante  H:  S16
Atlético  vs.  Getafe  H:  S18
Valencia  vs.  Real Sociedad  H:  S20
Espannyol  vs.  Sevilla  H:  D16
Real Madrid  vs.  Celta  H:  D18
Athletic  vs.  Barcelona  H:  D20
Betis  vs.  Valladolid  H:  L20
Mallorca  vs.  Eibar  H:  S20
Leganés  vs.  Osasuna  H:  D20
Audiencia:  6.33
Villareal  vs.  Granada  H:  V20
Athletic  vs.  Barcelona  H:  S16
Alavés  vs.  Levante  H:  S18
Valencia  vs.  Real Sociedad  H:  S20
Espannyol  vs.  Sevilla  H:  D16
Atlético  vs.  Getafe  H:  D18
Real Madrid  vs.  Celta  H:  D20
Betis  vs.  Valladolid  H:  L20
Leganés  vs.  Osasuna  H:  S16
Mallorca  vs.  Eibar 